In [ ]:
# Cell 1: Imports & basic config
import os, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from pennylane.templates.layers import StronglyEntanglingLayers
from sklearn.model_selection import train_test_split

import pennylane as qml

# Hyperparameters
N_QUBITS   = 6
FEAT_DIM   = 64     # 2**6
BATCH_SIZE = 32
EPOCHS     = 500
LR         = 1e-3
PRINT_EVERY = 1

DEVICE = torch.device("cpu") 
LOG_FILE = "output.txt"

assert FEAT_DIM == 2**N_QUBITS, "Input dimension must be 64 (=2**6)."


In [ ]:
# Cell 2: Load dataset
# Ensure the 'dataset' folder contains 'data.npy' and 'labels.npy'.

def load_data(folder_path):
    data = np.load(os.path.join(folder_path, 'data_speck.npy'))
    labels = np.load(os.path.join(folder_path, 'labels_speck.npy'))
    return data, labels

# Load data from the specified folder
datas, labels = load_data('dataset')

# Split dataset into training and validation sets (80/20 split)
train_data, val_data, train_labels, val_labels = train_test_split(
    datas, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")

# Prepare Torch Tensors & DataLoaders
Xtr = torch.from_numpy(train_data.astype(np.float32))
ytr = torch.from_numpy(train_labels.astype(np.int64))
Xva = torch.from_numpy(val_data.astype(np.float32))
yva = torch.from_numpy(val_labels.astype(np.int64))

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=BATCH_SIZE, shuffle=False)

data_size = len(Xtr) + len(Xva)

In [ ]:
# Cell 3: Quantum Circuit Definitions
# Defines the Convolutional and Pooling layers for the QCNN (Model A).

# Initialize PennyLane device
# Using 'lightning.qubit' for faster CPU simulation.
try:
    dev = qml.device("lightning.qubit", wires=N_QUBITS)
except Exception:
    dev = qml.device("default.qubit", wires=N_QUBITS)

def circuit8_block(thetas, wires):
    """
    Convolutional Block (Unitary).
    Applies parameterized rotations and entanglements.
    """
    a, b = wires
    # left locals
    qml.RZ(thetas[0], wires=a); qml.RZ(thetas[1], wires=b)
    qml.RY(thetas[2], wires=a); qml.RY(thetas[3], wires=b)
    qml.RZ(thetas[4], wires=a); qml.RZ(thetas[5], wires=b)

    qml.CNOT(wires=[a, b])

    # middle locals
    qml.RY(thetas[6], wires=a); qml.RZ(thetas[7], wires=b)

    qml.CNOT(wires=[b, a])

    qml.RY(thetas[8], wires=a)

    qml.CNOT(wires=[a, b])

    # right locals
    qml.RZ(thetas[9],  wires=a); qml.RZ(thetas[10], wires=b)
    qml.RY(thetas[11], wires=a); qml.RY(thetas[12], wires=b)
    qml.RZ(thetas[13], wires=a); qml.RZ(thetas[14], wires=b)

def pooling_block(phi2, wires, keep="left"):
    """
    Pooling Layer.
    Measures one qubit and controls the other to reduce dimensionality.
    """
    a, b = wires
    if keep == "left":
        drop, keepq = b, a
    else:
        drop, keepq = a, b
    qml.CRZ(phi2[0], wires=[drop, keepq])
    qml.CRX(phi2[1], wires=[drop, keepq])


@qml.qnode(dev, interface="torch", diff_method="best")
def qc_scouter_qnode(x,
                     weights,
                     theta_c1, 
                     theta_c2, 
                     theta_24,  
                     theta_02,  
                     phi_p1,     
                     phi_p2,     
                     phi_p3):    

    # Amplitude Embedding for input data
    qml.AmplitudeEmbedding(x, wires=range(6), normalize=True)
    
    # Strongly Entangling Layers (Initial Feature Map)
    qml.StronglyEntanglingLayers(weights, wires=range(6))
    
    # Convolutional Layer 1
    for a, b in [(0,1), (2,3), (4,5)]:
        circuit8_block(theta_c1, wires=[a,b])     
    
    # Convolutional Layer 2
    for a, b in [(1,2), (3,4), (5,0)]:
        circuit8_block(theta_c2, wires=[a,b])        

    # Pooling Layer 1
    pooling_block(phi_p1, wires=[0,1], keep="left") 
    pooling_block(phi_p1, wires=[2,3], keep="left")  
    pooling_block(phi_p1, wires=[4,5], keep="left") 

    # Deep Convolution & Pooling (Layer 3 & 4)
    circuit8_block(theta_24, wires=[2,4])
    pooling_block(phi_p2, wires=[2,4], keep="left")  

    circuit8_block(theta_02, wires=[0,2])
    pooling_block(phi_p3, wires=[0,2], keep="left")  

    # Measurement
    return qml.expval(qml.PauliZ(0))

In [ ]:
# Cell 4: QCScouter Model Definition

class QCScouter(nn.Module):
    def __init__(self):
        super().__init__()
        # Initialize parameters (Weights & Rotations)
        self.weights = nn.Parameter(0.01 * torch.randn((2,6,3)))
        self.theta_c1 = nn.Parameter(0.01 * torch.randn(15))
        self.theta_c2 = nn.Parameter(0.01 * torch.randn(15))
        self.theta_24 = nn.Parameter(0.01 * torch.randn(15))
        self.theta_02 = nn.Parameter(0.01 * torch.randn(15))
        self.phi_p1   = nn.Parameter(0.01 * torch.randn(2))
        self.phi_p2   = nn.Parameter(0.01 * torch.randn(2))
        self.phi_p3   = nn.Parameter(0.01 * torch.randn(2))

    def forward(self, x):
        # Handle 1D input case
        if x.dim() == 1:
            x = x.unsqueeze(0)
            
        z_list = []
        # Iterate over batch (Manual QNode execution per sample)
        for xi in x:
            z = qc_scouter_qnode(
                xi, self.weights,
                self.theta_c1, self.theta_c2,
                self.theta_24, self.theta_02,
                self.phi_p1,   self.phi_p2,   self.phi_p3
            )
            z_list.append(z)
            
        # Stack results and shape into logits for CrossEntropy
        z = torch.stack(z_list).unsqueeze(1)     
        logits = torch.cat([z, -z], dim=1)  # [z, -z] for 2-class classification      
        return logits

model = QCScouter() 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
print(model)


In [ ]:
# Cell 5: validate (simple)
@torch.no_grad()
def validate(model, loss_fn, val_loader):
    model.eval() # Switch to evaluation mode
    total = 0
    correct = 0
    loss_sum = 0.0
    
    for xb, yb in val_loader:
        xb = xb.to(DEVICE); yb = yb.to(DEVICE) # Move batch to device
        
        logits = model(xb) # Forward pass
        loss = loss_fn(logits, yb)
        loss_sum += loss.item() * xb.size(0) # Accumulate batch loss
        
        pred = torch.argmax(logits, dim=1) # Get predicted class indices
        correct += (pred == yb).sum().item() # Count correct predictions
        total += xb.size(0)
        
    # Return average loss and accuracy (with division safety)
    return (loss_sum / max(1,total)), (correct / max(1,total))


In [ ]:
# Cell 6: Training Loop & Logging (No Scheduler)

chart = []

# Log start time and dataset size
with open(LOG_FILE, "a") as f:
    f.write('\no ' + str(datetime.datetime.now()) + f' {data_size} samples (6q)\n')

for epoch in range(EPOCHS):
    model.train() # Set model to training mode
    last_train_loss = None

    for xb, yb in train_loader:
        xb = xb.to(DEVICE); yb = yb.to(DEVICE) # Move batch to device
        
        optimizer.zero_grad(set_to_none=True) # Reset gradients
        
        logits = model(xb) # Forward pass
        loss = loss_fn(logits, yb) # Compute loss
        loss.backward() # Backpropagation
        optimizer.step() # Update parameters
        
        last_train_loss = loss.item()

    # Perform validation
    val_loss, val_acc = validate(model, loss_fn, val_loader)

    # Format output string
    output = f"Epoch {epoch+1:3d} : Train Loss= {last_train_loss:.4f}, Validation Loss= {val_loss:.4f}, Accuracy= {val_acc:.4f}"
    chart.append(output)
    
    # Append to log file
    with open(LOG_FILE, "a") as f:
        f.write(output + '\n')
        
    # Print progress
    if (epoch+1) % PRINT_EVERY == 0:
        print(output)

# Final Logging: End time, Optimizer state, and Parameter counts
with open(LOG_FILE, "a") as f:
    f.write('~ ' + str(datetime.datetime.now()) + '\n' + str(optimizer) + '\n')
    f.write("Param Total " + str(sum(p.numel() for p in model.parameters())) + '\n')
    for name, p in model.named_parameters():
        f.write('    ' + str(name) + ' ' + str(p.numel()) + '\n')

print("Done. Logs ->", LOG_FILE)